In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch3/users/d/djl34"

In [2]:
client = Client()

In [16]:
df = dd.read_parquet(os.path.join(aso_data_dir, "whole_gene/ad/*/*parquet"))

In [18]:
misplicing_gain = df[(df["DS_AG"] > 0.1) | (df["DS_DG"] > 0.1)]


In [19]:
misplicing_gain = misplicing_gain.compute()

In [21]:
misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].replace(".", None)
misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].astype(float)

misplicing_gain["MaxEntScan_ref"] = misplicing_gain["MaxEntScan_ref"].replace(".", None)
misplicing_gain["MaxEntScan_ref"] = misplicing_gain["MaxEntScan_ref"].astype(float)

misplicing_gain = misplicing_gain[misplicing_gain["MaxEntScan_alt"] > 2]

In [22]:
misplicing_gain = misplicing_gain[(((misplicing_gain["DS_AL"] > 0.1) |(misplicing_gain["DS_DL"] > 0.1)) & 
                (misplicing_gain["MaxEntScan_alt"] < misplicing_gain["MaxEntScan_ref"]) & 
                ((misplicing_gain["MaxEntScan_alt"] < 2) | 
                 (misplicing_gain["MaxEntScan_alt"] < 0.3 * misplicing_gain["MaxEntScan_ref"]))) == False]

In [24]:
misplicing_gain_nodup = misplicing_gain.drop_duplicates(["CHROM", "Pos", "Allele"])

In [25]:
misplicing_gain_nodup

,CHROM,Pos,Allele_ref,Allele,Consequence,Gene,Feature,MaxEntScan_alt,MaxEntScan_diff,MaxEntScan_ref,LaBranchoR_start,LaBranchoR_score,spliceai_gene,DS,DS_AG,DS_AL,DS_DG,DS_DL
4952,1,155337765,C,T,splice_polypyrimidine_tract_variant&intron_var...,ENSG00000116539,ENST00000392403,4.679,3.491,8.169,NaN,NaN,ASH1L,0.64,0.18,0.64,0.00,0.00
4955,1,155337766,A,T,splice_polypyrimidine_tract_variant&intron_var...,ENSG00000116539,ENST00000392403,6.030,2.140,8.169,NaN,NaN,ASH1L,0.63,0.12,0.63,0.00,0.00
5866,1,155338070,T,C,intron_variant,ENSG00000116539,ENST00000392403,7.533,.,8.169,NaN,NaN,ASH1L,0.47,0.00,0.00,0.47,0.05
5873,1,155338072,A,T,intron_variant,ENSG00000116539,ENST00000392403,7.533,.,8.169,NaN,NaN,ASH1L,0.37,0.00,0.00,0.37,0.03
5908,1,155338084,C,G,splice_donor_5th_base_variant&intron_variant,ENSG00000116539,ENST00000392403,4.867,4.610,9.477,NaN,NaN,ASH1L,0.52,0.00,0.00,0.20,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110225,22,23826058,G,T,intron_variant,ENSG00000099956,ENST00000644036,10.280,.,9.786,NaN,NaN,SMARCB1,0.14,0.14,0.00,0.00,0.00
112552,22,23826834,C,G,intron_variant,ENSG00000099956,ENST00000644036,10.280,.,9.786,NaN,NaN,SMARCB1,0.45,0.00,0.00,0.45,0.00
114459,22,23827470,C,A,intron_variant,ENSG00000099956,ENST00000644036,10.280,.,9.786,NaN,NaN,SMARCB1,0.22,0.00,0.00,0.22,0.00
116698,22,23828216,C,G,intron_variant,ENSG00000099956,ENST00000644036,10.280,.,9.786,NaN,NaN,SMARCB1,0.15,0.00,0.00,0.15,0.00


In [26]:
exon_skipping = pd.read_csv(os.path.join(aso_data_dir, "whole_gene/ad_exon_skipping_2.tsv"), sep = "\t")

In [28]:
exon_skipping_nodup = exon_skipping.drop_duplicates(["CHROM", "Pos", "Allele"])

In [29]:
exon_skipping_nodup

,CHROM,Pos,Allele_ref,Allele,Consequence,Gene,Feature,MaxEntScan_alt,MaxEntScan_diff,MaxEntScan_ref,LaBranchoR_start,LaBranchoR_score,spliceai_gene,DS,DS_AG,DS_AL,DS_DG,DS_DL
0,1,155337702,T,A,synonymous_variant,ENSG00000116539,ENST00000392403,.,.,.,NaN,NaN,ASH1L,0.22,0.00,0.22,0.00,0.00
1,1,155337702,T,G,synonymous_variant,ENSG00000116539,ENST00000392403,.,.,.,NaN,NaN,ASH1L,0.12,0.00,0.12,0.00,0.00
2,1,155337703,C,A,missense_variant,ENSG00000116539,ENST00000392403,.,.,.,NaN,NaN,ASH1L,0.27,0.11,0.27,0.00,0.00
3,1,155337703,C,G,missense_variant,ENSG00000116539,ENST00000392403,.,.,.,NaN,NaN,ASH1L,0.13,0.01,0.13,0.00,0.00
4,1,155337703,C,T,missense_variant,ENSG00000116539,ENST00000392403,.,.,.,NaN,NaN,ASH1L,0.15,0.01,0.15,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365269,22,23825416,G,C,splice_donor_variant,ENSG00000099956,ENST00000644036,1.513,8.273,9.786,NaN,NaN,SMARCB1,1.00,0.00,0.00,0.01,1.00
365270,22,23825416,G,T,splice_donor_variant,ENSG00000099956,ENST00000644036,1.282,8.504,9.786,NaN,NaN,SMARCB1,1.00,0.00,0.00,0.32,1.00
365271,22,23825417,T,A,splice_donor_variant,ENSG00000099956,ENST00000644036,1.602,8.183,9.786,NaN,NaN,SMARCB1,1.00,0.00,0.00,0.01,1.00
365272,22,23825417,T,C,splice_donor_variant,ENSG00000099956,ENST00000644036,2.032,7.754,9.786,NaN,NaN,SMARCB1,0.56,0.00,0.00,0.01,0.56
